In [1]:
from QMLE_scipy import *
from explore import *

### 1. Test k=2

In [2]:
# set up initial guess
sigma = 1
lam = 0.5
gamma = 0.1
rho = 0.2
beta = np.array([1,2])
params = [sigma,lam,gamma, rho, *beta]
params

# set up input data
n = 4
T = 4
k = 2
alpha = np.random.normal(0,1,4)
c0 = np.random.normal(0,1,4)
x1 = []
for i in range(T):
    x1.append(np.random.normal(0,1,n*k).reshape(n,k))
y1 = []
for i in range(T+1):
    y1.append(np.random.normal(0,1,4))
# manually setting up a row-normalized spatial weight vector with 0 diagonals for even time points
w0 = np.array([[0,1,0,0],[1,0,0,0],[0,0,0,1],[0,1,0,0]]).reshape(4,4)
w1 = np.array([[0,0,1,0],[1,0,0,0],[0,1,0,0],[1,0,0,0]]).reshape(4,4)
W_ls = []
for i in range(T+1):
    if is_even(i):
        W_ls.append(w0)
    else:
        W_ls.append(w1)

In [4]:
# run optimization with constraints
result = QMLE_scipy_estimate(x1, y1, W_ls,initial_guess = params,est_coeff = False)

In [6]:
result.params

[0.6572670293395565,
 0.005654012937840697,
 -0.8010357962132825,
 -0.3985381230498649,
 array([-0.04058268, -0.04617951])]

In [7]:
result.gamma()

-0.8010357962132825

In [8]:
result.residual_mean()

0.190227706563247

In [9]:
result.residual_std()

0.7738978614759432

In [10]:
result.print_table()

Residuals:
     Min         1Q    Median        3Q      Max
--------  ---------  --------  --------  -------
-1.14437  -0.218775  0.157799  0.694451  1.69913

Estimated Parameters:
   Sigma      Lambda      Gamma        Rho  Beta
--------  ----------  ---------  ---------  -------------------------
0.657267  0.00565401  -0.801036  -0.398538  [-0.04058268 -0.04617951]

Estimated alpha: 
  alpha_0    alpha_1    alpha_2    alpha_3
---------  ---------  ---------  ---------
0.0103141  0.0395066  -0.477688   0.427868

Estimated c0 (display first 10 rows): 
    c0_0      c0_1        c0_2      c0_3
--------  --------  ----------  --------
0.829268  0.687323  -0.0345317  0.741833

Estimated asympotitic variance: 
               Gamma         Rho      Beta_0       Beta_1      Lambda      Sigma^2
-------  -----------  ----------  ----------  -----------  ----------  -----------
Gamma     0.0960397    0.0158824   0.0330353   0.00522544  -0.0188126  -0.00717179
Rho       0.0158824    1.04192     0

In [11]:
result = QMLE_scipy_estimate(x1, y1, W_ls,initial_guess = params, constrain= False)

In [12]:
result.print_table()

Residuals:
     Min         1Q    Median        3Q      Max
--------  ---------  --------  --------  -------
-1.14437  -0.218777  0.157799  0.694462  1.69913

Estimated Parameters:
   Sigma      Lambda      Gamma        Rho  Beta
--------  ----------  ---------  ---------  -------------------------
0.657267  0.00565014  -0.801035  -0.398541  [-0.04058327 -0.04618029]

Estimated alpha: 
  alpha_0    alpha_1    alpha_2    alpha_3
---------  ---------  ---------  ---------
0.0103123  0.0395061  -0.477691   0.427873

Estimated c0 (display first 10 rows): 
    c0_0      c0_1        c0_2      c0_3
--------  --------  ----------  --------
0.829265  0.687327  -0.0345315  0.741833

Estimated asympotitic variance: 
               Gamma         Rho      Beta_0       Beta_1      Lambda     Sigma^2
-------  -----------  ----------  ----------  -----------  ----------  ----------
Gamma     0.0960396    0.0158819   0.0330352   0.00522553  -0.0188131  -0.007172
Rho       0.0158819    1.04191     0.226

### 2. Test k=1

In [13]:
# set up initial guess
initial_guess = [1.5,0.2,0.5,0.3,1.2] # in the order sigma, lam, gamma, rho, beta (all real numbers)

n = 20
k = 1
T = 10

# generate samples
t_theta = [0.1,0.2,np.array([1]).reshape(k,1),0.5,1] # true parameters
t_gamma, t_rho, t_beta, t_lam, t_sig = t_theta
alpha = np.random.normal(0,1,T).reshape(T,1)
c0 = np.random.normal(0,1,n).reshape(n,1)
l_n = np.ones(n).reshape(n,1)
x = []
for i in range(T):
    x.append(np.random.normal(0,1,n).reshape(n,k))
# manually setting up a row-normalized spatial weight vector with 0 diagonals for even time points
w0 = generate_weight_matrix(n)
w1 = generate_weight_matrix(n)

W_ls = []
for i in range(T+1):
    if is_even(i):
        W_ls.append(w0)
    else:
        W_ls.append(w1)

        V_nt = []
for i in range(T):
    V_nt.append(np.random.normal(0,t_sig,n).reshape(n,1))

Y0 = np.random.normal(0,1,n).reshape(n,1)

y = []
y.append(Y0)
for i in range(T):
    c_vec = t_gamma*y[i]+t_rho*np.matmul(W_ls[i],np.array(y[i]).reshape(n,1))+np.matmul(x[i],np.array(t_beta).reshape(k,1))+c0+alpha[i][0]*l_n+V_nt[i]
    Y_nt = np.matmul(np.linalg.inv(np.identity(n)-t_lam*W_ls[i+1]),c_vec).reshape(n,1)
    y.append(Y_nt)

In [14]:
result = QMLE_scipy_estimate(x, y, W_ls,initial_guess = initial_guess, constrain= False)

In [15]:
result.print_table()

Residuals:
     Min         1Q      Median       3Q      Max
--------  ---------  ----------  -------  -------
-3.20638  -0.895513  -0.0683062  0.50206  2.84372

Estimated Parameters:
   Sigma    Lambda      Gamma       Rho      Beta
--------  --------  ---------  --------  --------
0.904726  0.496518  0.0268611  0.226456  0.839046

Estimated alpha: 
  alpha_0    alpha_1      alpha_2    alpha_3     alpha_4    alpha_5    alpha_6    alpha_7     alpha_8    alpha_9
---------  ---------  -----------  ---------  ----------  ---------  ---------  ---------  ----------  ---------
-0.656302   0.328088  -0.00884961  -0.144737  -0.0575587   0.709578   -1.02981   0.336587  -0.0278937   0.550901

Estimated c0 (display first 10 rows): 
   c0_0      c0_1     c0_2       c0_3       c0_4      c0_5      c0_6     c0_7       c0_8       c0_9
-------  --------  -------  ---------  ---------  --------  --------  -------  ---------  ---------
0.37483  0.504901  1.16341  -0.912153  -0.462692  -0.58143  0.428066

In [16]:
result = QMLE_scipy_estimate(x, y, W_ls,initial_guess = initial_guess, constrain= False)

In [17]:
result.print_table()

Residuals:
     Min         1Q      Median       3Q      Max
--------  ---------  ----------  -------  -------
-3.20638  -0.895513  -0.0683062  0.50206  2.84372

Estimated Parameters:
   Sigma    Lambda      Gamma       Rho      Beta
--------  --------  ---------  --------  --------
0.904726  0.496518  0.0268611  0.226456  0.839046

Estimated alpha: 
  alpha_0    alpha_1      alpha_2    alpha_3     alpha_4    alpha_5    alpha_6    alpha_7     alpha_8    alpha_9
---------  ---------  -----------  ---------  ----------  ---------  ---------  ---------  ----------  ---------
-0.656302   0.328088  -0.00884961  -0.144737  -0.0575587   0.709578   -1.02981   0.336587  -0.0278937   0.550901

Estimated c0 (display first 10 rows): 
   c0_0      c0_1     c0_2       c0_3       c0_4      c0_5      c0_6     c0_7       c0_8       c0_9
-------  --------  -------  ---------  ---------  --------  --------  -------  ---------  ---------
0.37483  0.504901  1.16341  -0.912153  -0.462692  -0.58143  0.428066